In [1]:
import functools
import numpy as np
import os.path
import scipy.linalg as sla
import sys
import datetime
import os
import psutil

from dask.distributed import Client
import dask.array as da

from sklearn.datasets import make_sparse_coded_signal

In [2]:
#Inputs
'''
These inputs were copied from the pyspark version.
I may not need all of these.

'''


input_data = ''
T = 20 #Rows
P = 20 #Cols

#Optional
R = int(0.07 * P) #Enforces sparcity 
M = 5 #Dimensionality of the learned dictionary
E = 0.01 #Epsilon
norm = True


#Spark Options, look for similar Dask elements
partitions = 4*4
execmem = '8g'

#Outputs
d = ''
z = ''
prefix = False

In [3]:
client = Client()

In [21]:
S, D, SV = make_sparse_coded_signal(M,T,P,R)

u_new = da.zeros(T).compute()
v = da.zeros(P).compute()

max_iterations = P*10

print(S)

[[ 0.06541856 -0.01946797 -0.19816472  0.1204741  -0.05476803]
 [-0.01677284 -0.01023524 -0.0511474  -0.03088868 -0.12316574]
 [ 0.05620451 -0.32736995 -0.55464978  0.10350562  0.12117533]
 [-0.08586406 -0.25118923 -0.01001002 -0.15812631 -0.05461503]
 [-0.17274095 -0.14297508 -0.08418071 -0.31811786  0.08090529]
 [ 0.05598561 -0.18824401  0.36425091  0.10310248  0.10118473]
 [ 0.10419019 -0.15751331  0.20441857  0.19187551  0.03198287]
 [-0.01118214 -0.12976929  0.1937893  -0.0205929  -0.0323587 ]
 [-0.17529397  0.06444068  0.08298722 -0.32281946  0.01343229]
 [ 0.09539643  0.19679415 -0.23937345  0.17568102 -0.09018886]
 [-0.08197947 -0.42471692 -0.11476712 -0.15097249 -0.05814707]
 [-0.26226932  0.21752597 -0.07321165 -0.48299233 -0.14337092]
 [-0.22091983 -0.08176746  0.09546011 -0.40684354  0.13459982]
 [-0.17136799  0.59922849  0.06082647 -0.31558942 -0.08509391]
 [-0.14352068 -0.10435476 -0.76560762 -0.26430612 -0.19886068]
 [-0.12214011  0.00344286  0.45390989 -0.22493189  0.02

In [25]:
for m in range(M):
    #We need to broadcast the seed to create the initial random vector 
    seed = np.random.randint(max_iterations +1, high = 4294967295)
    _SEED_ = client.scatter(seed, broadcast=True)
    np.random.seed(_SEED_.result())
    
    #Create a dense random vector
    u_old = da.random.random(T).compute()
    num_iterations = 0
    delta = 2*E
    print(u_old)
    
    while num_iterations < max_iterations and delta > E:
        
        _U_ = client.scatter(u_old, broadcast=True) 
            
        v = da.dot(_U_.result(),S).compute() #Can I delay? 
        print(f'v:\t{v}')
        
        '''How to find the indicies and which sparse implimentation is correct'''
        v = np.take(sorted(v), indices = 1, axis = 1) #I dont think this is neccessary?

        # Use our previous method to select the top R.
        indices = np.sort(select_topr(v, R))
        sv = SparseVector(P, indices, v[indices]) #Still need to find a solution!!! 
        '''End of sparse implimentation DONT FORGET TO UPDATE WHEN WRITING TO FILE'''
        
        # Broadcast the sparse vector.
        _V_ = client.scatter(sv,broadcast=True)

        # P1: Matrix-vector multiplication step. Computes u.
        u_new = da.dot(S,_V_.result())
        u_new = np.take(sorted(u_new), indices = 1, axis = 1)

        # Subtract off the mean and normalize.
        u_new -= u_new.mean()
        u_new /= da.linalg.norm(u_new)
        u_new.compute()
        
        # Update for the next iteration.
        delta = da.linalg.norm(u_old - u_new).compute() #Should u_old be _U_?
        u_old = u_new
        num_iterations += 1
    
    # Save the newly-computed u and v to the output files;
    with open(file_D, "a+") as fD:
        np.savetxt(fD, u_new, fmt = "%.6f", newline = " ")
        fD.write("\n")
    with open(file_z, "a+") as fz:
        np.savetxt(fz, sv, fmt = "%.6f", newline = " ")
        fz.write("\n")

    # P4: Deflation step. Update the primary data matrix S.
    _U_ = client.scatter(u_new, broadcast=True)
    _V_ = client.scatter(sv, broadcast=True)

    if args['debug']: print(m)

    S -= da.outer(_U_.result(),_V_.result())

    if args['debug']: print(datetime.datetime.now())
    process = psutil.Process(os.getpid())
    print(process.memory_info().rss)

[0.45542876 0.76055827 0.40946065 0.82122679 0.97121719 0.51038366
 0.32310577 0.49026075 0.97722678 0.40832227 0.08477151 0.16254759
 0.75126085 0.34550614 0.6503721  0.70943455 0.11521253 0.94471443
 0.2108304  0.29277693]
v:	[-0.51937104  0.00552614 -0.74034385 -0.95646805  0.0569729 ]


AxisError: axis 1 is out of bounds for array of dimension 1

In [15]:
import sparse

In [24]:
x = da.random.random((100000,100000))
x[x<0.95] = 0
s = x.map_blocks(sparse.COO).compute()

In [21]:
s

dask.array<COO, shape=(100000, 100000), dtype=float64, chunksize=(4000, 4000), chunktype=sparse.COO>